# Variational Quantum Simulators

![image.png](images/fullstack.png)

![image.png](images/zygote-yao.png)

![image.png](images/mlqc_review.png)

![image.png](images/projectq_github.png)

# Variational Quantum Softwares

![image.png](images/pennylane_github.png)

![image.png](images/qulacs_github.png)

![image.png](images/quest_github.png)

## Open source software in quantum computing
https://arxiv.org/abs/1812.09167
## Making Quantum Computing Open: Lessons from Open-Source Projects
https://arxiv.org/abs/1902.00991

# A benchmark
![benchmark](images/qulacs_benchmark.png)

# Yao framework
![yaoarch](images/yaoarch.png)

# QBIR Example: Dagger

In [ ]:
Dagger

In [5]:
using Yao, BitBasis

In [6]:
# time evolution

In [12]:
reg = product_state(bit"010")
statevec(reg)

8-element Array{Complex{Float64},1}:
 0.0 + 0.0im
 0.0 + 0.0im
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [16]:
swap21 = swap(4,2,1)

nqubits: 4
put on (2, 1)
└─ SWAP gate

In [22]:
statevec(reg |> swap21)

8-element Array{Complex{Float64},1}:
  0.0 + 0.0im
 -0.0 - 1.0im
  0.0 + 0.0im
  0.0 + 0.0im
  0.0 + 0.0im
  0.0 + 0.0im
  0.0 + 0.0im
  0.0 + 0.0im

# why Julia

# **Performance**
Solving two language problem

* **ProjectQ**: C++ (pybind) python
* **Quest**: Pure C
* **Qulacs**: C++ (and) python

In [8]:
# When does two language problem becomes a problem
# 2-qubit gates
using YaoBase
using YaoArrayRegister
function YaoBase.instruct!(
        state::AbstractVecOrMat{T},
        ::Val{:SWAP},
        locs::Tuple{Int, Int}) where T

    mask1 = bmask(locs[1])
    mask2 = bmask(locs[2])
    mask12 = mask1|mask2
    for b in basis(state)
        if b&mask1==0 && b&mask2==mask2
            i = b+1
            i_ = b ⊻ mask12 + 1
            YaoArrayRegister.swaprows!(state, i, i_)
        end
    end
    state .*= im
    return state
end

In [24]:
statevec(reg |> swap21)

8-element Array{Complex{Float64},1}:
 0.0 + 0.0im
 0.0 + 1.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

# Strong community
* *JuliaGPU*

An example of GPU swap gate

In [ ]:
function instruct!(reg::ArrayReg{B,<:CuArrays}, ::Val{:SWAP}, locs::Tuple{Int,Int}) where {B}
    b1, b2 = locs
    state = statevec(reg)
    mask1 = bmask(b1)
    mask2 = bmask(b2)

    # note: this kernel does not have the best performance,
    # shared memory + `shfl_xor` will probably increase the memory access performance.
    function kf(state, mask1, mask2)
        inds = ((blockIdx().x-1) * blockDim().x + threadIdx().x,
        (blockIdx().y-1) * blockDim().y + threadIdx().y)
        b = inds[1]-1
        c = inds[2]
        c <= size(state, 2) || return nothing
        if b&mask1==0 && b&mask2==mask2
            i = b+1
            i_ = b (mask1|mask2) + 1
            temp = state[i, c]
            state[i, c] = state[i_, c]
            state[i_, c] = temp
        end
        nothing
    end
    X, Y = cudiv(size(state)...)
    @cuda threads=X blocks=Y kf(state, mask1, mask2)
    state
end

# No worry, we have CuYao

In [ ]:
reg = rand_state(20)
@benchmark reg |> swapgate

In [ ]:
@benchmark cureg |> swapgate

In [ ]:
# if we have batch

# Automatic Differentiation